## Calculating focal plane offsets

Craig Lage - 10-May-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from matplotlib.patches import Rectangle as Rect
from lsst.summit.utils.efdUtils import calcNextDay
import pickle as pkl
from astropy.coordinates import angular_separation
from scipy.optimize import curve_fit
import lsst.geom as geom


In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157',
                                          "LSSTCam/raw/guider"])
instrument = 'LSSTCam'   

In [ ]:
def get_offsets(butler, camera, expId, ref_detName, img_detName):
    # Gets the offsets from one detector to another based on astrometry
    ref_detector = camera.get(ref_detName)
    ref_calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
    ref_wcs = ref_calexp.getWcs()
    ref_bbox = ref_detector.getBBox()
    ref_center_pixels = Point2D(ref_bbox.centerX, ref_bbox.centerY)
    ref_center = ref_wcs.pixelToSky(ref_center_pixels)
    img_detector = camera.get(img_detName)
    img_calexp = butler.get('preliminary_visit_image', detector=img_detector.getId(), visit=expId)
    img_wcs = img_calexp.getWcs()
    img_bbox = img_detector.getBBox()
    img_center_pixels = Point2D(img_bbox.centerX, img_bbox.centerY)
    img_center = img_wcs.pixelToSky(img_center_pixels)
    img_center_ref = ref_wcs.skyToPixel(img_center)
    delta_center = img_center_ref - ref_center_pixels
    delta_center / 100.0 #convert to mm
    rot = img_wcs.getRelativeRotationToWcs(ref_wcs).asDegrees()
    return delta_center, rot


In [ ]:
get_offsets(butler, camera, expId, 'R22_S11', 'R22_S12')

In [ ]:
def draw_raft_with_offsets(butler, camera, raft, file):
    ccds = ['S11', 'S00', 'S01', 'S02', 'S10', 
            'S12', 'S20', 'S21', 'S22']
    fig, ax = plt.subplots(1,1,figsize=(10,10))
    ax.grid(False)
    plt.axis('off') 
    ax.set_aspect(1)
    ax.set_title(f"{raft} offsets in mm, {expId}\nBlack:design, Red:Astrometry")
    ref_detName = 'R22_S11'
    ref_detector = camera.get(ref_detName)
    llX = 100000.0; llY = 100000.0; urX = -100000.0; urY = -100000.0
    for ccd in ccds:
        img_detName = raft + '_' + ccd
        file.write(img_detName)
        img_detector = camera.get(img_detName)
        img_bbox = img_detector.getBBox()
        lct = LsstCameraTransforms(camera,img_detName)
        llfpX,llfpY = lct.ccdPixelToFocalMm(img_bbox.beginX, img_bbox.beginY, img_detName)
        urfpX,urfpY = lct.ccdPixelToFocalMm(img_bbox.endX, img_bbox.endY, img_detName)
        if llfpX < llX:
            llX = llfpX
        if llfpY < llY:
            llY = llfpY
        if urfpX > urX:
            urX = urfpX
        if urfpY > urY:
            urY = urfpY

        cenfpX,cenfpY = lct.ccdPixelToFocalMm(img_bbox.centerX, img_bbox.centerY, img_detName)
        rect = Rect((llfpX, llfpY), img_bbox.getDimensions().getX() / 100,
                             img_bbox.getDimensions().getY() / 100, edgecolor='k', facecolor='none')
        ax.add_patch(rect)
        ax.scatter (cenfpX, cenfpY, marker='+', color='k')
        ax.text(cenfpX, urfpY - 4.0, img_detName, ha='center', va='baseline')
        design_location = f"{cenfpX:.3f}, {cenfpY:.3f}"
        ax.text(cenfpX, cenfpY + 8.0, design_location, ha='center', va='baseline')
        delta_center, rot = get_offsets(butler, camera, expId, ref_detName, img_detName)
        if rot > 180.0:
            rot = rot - 360.0
        measured_location = f"{delta_center.getX()/100:.3f}, {delta_center.getY()/100:.3f}"
        if ccd == 'S11':
            s11_center = delta_center
            s11_cenfpX = cenfpX
            s11_cenfpY = cenfpY
        ax.text(cenfpX, cenfpY + 4.0, measured_location, ha='center', va='baseline', color='red')
        diff_s11X = delta_center.getX()/100-s11_center.getX()/100
        diff_s11Y = delta_center.getY()/100-s11_center.getY()/100
        diff_s11 = f"Relative to S11: {diff_s11X:.3f}, {diff_s11Y:.3f}"
        ax.text(cenfpX, cenfpY - 4.0, diff_s11, ha='center', va='baseline', color='red')
        diff_designX = (delta_center.getX()/100-s11_center.getX()/100) - \
                        (cenfpX - s11_cenfpX)
        diff_designY = (delta_center.getY()/100-s11_center.getY()/100) - \
                        (cenfpY - s11_cenfpY)
        diff_design = f"Within raft offset: {diff_designX:.3f}, {diff_designY:.3f}"
        ax.text(cenfpX, cenfpY - 8.0, diff_design, ha='center', va='baseline', color='red')
        ax.text(cenfpX, cenfpY - 12.0, f"Rot={rot:.4f} degrees", ha='center', va='baseline', color='red')
        file.write(f"  {cenfpX:.3f}, {cenfpY:.3f}, {delta_center.getX()/100:.3f}, {delta_center.getY()/100:.3f} {rot:.4f}\n")
    ax.set_xlim(llX-1.0, urX+1.0)
    ax.set_ylim(llY-1.0, urY+1.0)

    return fig

In [ ]:
expId = 2025042500591
camera = LsstCam.getCamera()
ref_detName = 'R22_S11'
outfile = open (f"/home/c/cslage/u/LSSTCam/data/All_Rafts_Design_and_Measured_Offsets_{expId}.txt", 'w')

for img_detector in camera:
    try:
        img_detName = img_detector.getName()
        if img_detector.getId() > 188:
            continue
        delta_center, rot = get_offsets(butler, camera, expId, ref_detName, img_detName)
        if rot > 180.0:
            rot = rot - 360.0
        img_bbox = img_detector.getBBox()
        cenfpX,cenfpY = lct.ccdPixelToFocalMm(img_bbox.centerX, img_bbox.centerY, img_detName)
        outfile.write(img_detName)
        outfile.write(f"  {cenfpX:.3f}, {cenfpY:.3f}, {delta_center.getX()/100:.3f}, {delta_center.getY()/100:.3f} {rot:.4f}\n")
    except:
        continue
outfile.close()



In [ ]:
expId = 2025042500591
camera = LsstCam.getCamera()

raft = 'R33'
plt.clf()
fig = draw_raft_with_offsets(butler, camera, raft)
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/{raft}_Design_and_Measured_Offsets_{expId}.png")

## Look at offsets over a range of dates.

In [ ]:
camera = LsstCam.getCamera()
ref_detName = 'R22_S11'
img_detName = 'R33_S11'

startDay = 20250415
endDay = 20250506

offsets = {}
dayObs = startDay
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(dayObs, len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science']:
            continue
        try:
            delta_center, rot = get_offsets(butler, camera, id, ref_detName, img_detName)
            offsets[id] = delta_center
            print(f"{id} passed!")
        except:
            continue
            print(f"{id} failed!")
    print(f"{dayObs} complete.")
    dayObs = calcNextDay(dayObs)


filename = f"/home/c/cslage/u/LSSTCam/data/Offsets_{ref_detName}_{img_detName}_12May25.pkl"
with open(filename, 'wb') as f:
    pkl.dump(offsets, f)


## Now plot the histograms

In [ ]:
img_detector = camera.get(img_detName)
img_bbox = img_detector.getBBox()
lct = LsstCameraTransforms(camera,img_detName)
cenfpX,cenfpY = lct.ccdPixelToFocalMm(img_bbox.centerX, img_bbox.centerY, img_detName)
print(cenfpX, cenfpY)
offXs = []
offYs = []
for key in offsets.keys():
    offXs.append(offsets[key][0] / 100.0)
    offYs.append(offsets[key][1] / 100.0)

fig, axs = plt.subplots(1,2,figsize=(10,4))
plt.suptitle(f"Astrometric offset {img_detName} to {ref_detName}, Data from {startDay} to {endDay}")
axs[0].set_title("X offset (mm)")
axs[0].hist(offXs, bins=100)
axs[0].axvline(cenfpX, color='k', ls='--')
axs[0].set_xlim(126.90, 127.10)
axs[0].set_xticks([126.90, 127.00, 127.10])
axs[0].text(127.02, 400, f"Npoints = {len(offXs)}\n Delta = {(np.median(offXs) - cenfpX):.3f} mm.")
axs[1].set_title("Y offset (mm)")
axs[1].hist(offYs, bins=100)
axs[1].axvline(cenfpY, color='k', ls='--')
axs[1].set_xlim(126.90, 127.10)
axs[1].set_xticks([126.90, 127.00, 127.10])
axs[1].text(127.02, 1200, f"Npoints = {len(offYs)}\n Delta = {(np.median(offYs) - cenfpY):.3f} mm.")
plt.savefig(f"/home/c/cslage/u/LSSTCam/data/Offset_Histograms_{ref_detName}_{img_detName}_12May25.png")

## What about change in platescale?

In [ ]:
expId = 2025042500591
camera = LsstCam.getCamera()

xs = []
ys = []
scales = []
scale_dict = {}
for detector in camera:
    if detector.getId() > 188:
        continue
    try:
        detName = detector.getName()
        calexp = butler.get('preliminary_visit_image', detector=detector.getId(), visit=expId)
        wcs = calexp.getWcs()
        bbox = detector.getBBox()
        lct = LsstCameraTransforms(camera,detName)
        cenfpX,cenfpY = lct.ccdPixelToFocalMm(bbox.centerX, bbox.centerY, detName)
        xs.append(cenfpX)
        ys.append(cenfpY)
        scale = wcs.getPixelScale().asDegrees() * 3600
        print(detName, wcs.getPixelOrigin())
        scales.append(scale)
        scale_dict[detName] = [(cenfpX, cenfpY), scale]
    except:
        print(f"Detector {detName} failed!")
        continue

In [ ]:
from scipy.optimize import curve_fit

rs = np.sqrt(np.array(xs)**2 + np.array(xs)**2)
fig = plt.figure(figsize=(8,5))
plt.scatter(rs, scales)
def func(x, quad, const):
    # Quadratic function with no linear term
    # So df/dr(r=0) = 0.
    return quad * x * x + const
popt, pcov = curve_fit(func, rs, scales, p0=[1E-4, 0.0])
print(popt)
R22_S11_R33_S11_delta = ((127.0 * 100 * func(127.0 * np.sqrt(2.0), popt[0], popt[1])) - \
                         (127.0 * 100 * func(0.0 * np.sqrt(2.0), popt[0], popt[1]))) / np.median(scales) / 100.0
xplot = np.linspace(0,500, 100)
yplot = xplot * xplot * popt[0] + popt[1]

plt.plot(xplot, yplot, color='red', ls='--')
plt.title(f"Plate scale vs detector radius {expId}")
plt.xlabel("Radius to center of detector (mm)")
plt.ylabel("Platescale (arcseconds/pixel)")
plt.text(0, 0.20018, f"R22_S11 to R33_S11 delta = {(R22_S11_R33_S11_delta):.4f} mm")
plt.savefig(f"/home/c/cslage/u/LSSTCam/data/Platescale_Variation_{expId}_13May25.png")

In [ ]:
plt.tricontourf(xs, ys, scales)
plt.colorbar()
plt.title(f"Plate scale across focal plane {expId}")
plt.xlabel("X (mm)")
plt.ylabel("Y (mm)")
plt.savefig(f"/home/c/cslage/u/LSSTCam/data/Platescale_Contour_Plot_{expId}_13May25.png")

In [ ]:
expId = 2025042500591
camera = LsstCam.getCamera()

xs = []
ys = []
scales = []
scale_dict = {}
for detector in camera:
    if detector.getId() > 188:
        continue
    try:
        detName = detector.getName()
        calexp = butler.get('preliminary_visit_image', detector=detector.getId(), 
                            visit=expId, instrument=instrument)
        wcs = calexp.getWcs()
        bbox = detector.getBBox()
        lct = LsstCameraTransforms(camera,detName)
        cenfpX,cenfpY = lct.ccdPixelToFocalMm(bbox.centerX, bbox.centerY, detName)
        xs.append(cenfpX)
        ys.append(cenfpY)
        scale = wcs.getPixelScale(Point2D(bbox.centerX, bbox.centerY)).asArcseconds()
        scales.append(scale)
        scale_dict[detName] = [(cenfpX, cenfpY), scale]
    except:
        print(f"Detector {detName} failed!")
        continue

In [ ]:
rs = np.sqrt(np.array(xs)**2 + np.array(ys)**2)
fig = plt.figure(figsize=(8,5))
plt.scatter(rs, scales)
def func(x, quad, const):
    # Quadratic function with no linear term
    # So df/dr(r=0) = 0.
    return quad * x * x + const
popt, pcov = curve_fit(func, rs, scales, p0=[1E-4, 0.0])
print(popt)
R22_S11_R33_S11_delta = ((127.0 * 100 * func(127.0 * np.sqrt(2.0), popt[0], popt[1])) - \
                         (127.0 * 100 * func(0.0 * np.sqrt(2.0), popt[0], popt[1]))) / np.median(scales) / 100.0
xplot = np.linspace(0,350, 100)
yplot = xplot * xplot * popt[0] + popt[1]

plt.plot(xplot, yplot, color='red', ls='--')
plt.title(f"Plate scale vs detector radius {expId}")
plt.xlabel("Radius to center of detector (mm)")
plt.ylabel("Platescale (arcseconds/pixel)")
plt.text(0, 0.1997, f"R22_S11 to R33_S11 delta = {(R22_S11_R33_S11_delta):.4f} mm")
plt.savefig(f"/home/c/cslage/u/LSSTCam/data/Platescale_Variation_{expId}_With_X0_Y0_13May25.png")

In [ ]:
expId = 2025042500591
camera = LsstCam.getCamera()
calexp = butler.get('preliminary_visit_image', detector=94, visit=expId)
wcs = calexp.getWcs()

xs = []
ys = []
scales = []
scale_dict = {}
for detector in camera:
    if detector.getId() > 188:
        continue
    try:
        detName = detector.getName()
        bbox = detector.getBBox()
        lct = LsstCameraTransforms(camera,detName)
        cenfpX,cenfpY = lct.ccdPixelToFocalMm(bbox.centerX, bbox.centerY, detName)
        xs.append(cenfpX)
        ys.append(cenfpY)
        scale = wcs.getPixelScale(Point2D(bbox.centerX, bbox.centerY)).asArcseconds()
        scales.append(scale)
        scale_dict[detName] = [(cenfpX, cenfpY), scale]
        #print(detName, scale)
    except:
        print(f"Detector {detName} failed!")
        continue

In [ ]:
from scipy.optimize import curve_fit

rs = np.sqrt(np.array(xs)**2 + np.array(ys)**2)
fig = plt.figure(figsize=(8,5))
plt.scatter(rs, scales)
def func(x, quad, const):
    # Quadratic function with no linear term
    # So df/dr(r=0) = 0.
    return quad * x * x + const
popt, pcov = curve_fit(func, rs, scales, p0=[1E-4, 0.0])
print(popt)
R22_S11_R33_S11_delta = ((127.0 * 100 * func(127.0 * np.sqrt(2.0), popt[0], popt[1])) - \
                         (127.0 * 100 * func(0.0 * np.sqrt(2.0), popt[0], popt[1]))) / np.median(scales) / 100.0
xplot = np.linspace(0,350, 100)
yplot = xplot * xplot * popt[0] + popt[1]

plt.plot(xplot, yplot, color='red', ls='--')
plt.title(f"Plate scale vs detector radius {expId}")
plt.xlabel("Radius to center of detector (mm)")
plt.ylabel("Platescale (arcseconds/pixel)")
#plt.text(0, 0.1997, f"R22_S11 to R33_S11 delta = {(R22_S11_R33_S11_delta):.4f} mm")
plt.savefig(f"/home/c/cslage/u/LSSTCam/data/Platescale_Variation_{expId}_Center_WCS_15May25.png")

## How does effective plate scale vary with radius?

In [ ]:
expId = 2025042500591
ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera.get(ref_detName)
ref_calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), 
                        visit=expId, instrument=instrument)
ref_wcs = ref_calexp.getWcs()
ref_bbox = ref_detector.getBBox()
ref_center_pixels = Point2D(ref_bbox.centerX, ref_bbox.centerY)

x_shifts = np.linspace(0, 350, 50) # in mm
eff_scales = []
dx = 20 # +/- shift in pixels
for x_shift in x_shifts:
    pixel_shift = x_shift * 100 # Convert mm to pixels
    pixels_1 = Point2D(ref_bbox.centerX + pixel_shift - dx, ref_bbox.centerY)
    pixels_2 = Point2D(ref_bbox.centerX + pixel_shift + dx, ref_bbox.centerY)
    sky_1 = ref_wcs.pixelToSky(pixels_1)
    sky_2 = ref_wcs.pixelToSky(pixels_2)
    separation = np.degrees(angular_separation(sky_1.getRa().asRadians(), sky_1.getDec().asRadians(),
                                sky_2.getRa().asRadians(), sky_2.getDec().asRadians())) * 3600
    eff_scale = separation / (2.0 * dx)
    print(x, separation, eff_scale)
    eff_scales.append(eff_scale)
                            


In [ ]:
fig = plt.figure(figsize=(8,5))
plt.scatter(x_shifts, eff_scales, label='Effective plate scale from R22_S11 WCS')
def func(x, quad, const):
    # Quadratic function with no linear term
    # So df/dr(r=0) = 0.
    return quad * x * x + const
popt, pcov = curve_fit(func, x_shifts, eff_scales, p0=[1E-4, 0.0])
print(popt)
xplot = np.linspace(0,350, 100)
yplot = xplot * xplot * popt[0] + popt[1]

plt.plot(xplot, yplot, color='red', ls='--', label='Fit to  effective plate scales')
plt.scatter(rs, scales, marker='x', color='green', label='Plate scales from WCS for each CCD')
plt.title(f"Effective Plate scale vs detector radius {expId}")
plt.xlabel("Radius to center of detector or evaluation point (mm)")
plt.ylabel("Effective Platescale (arcseconds/pixel)")
plt.legend(loc='lower left')
plt.savefig(f"/home/c/cslage/u/LSSTCam/data/Platescale_Effective_Variation_{expId}_23May25.png")

In [ ]:
sky0 = ref_wcs.pixelToSky(ref_bbox.centerX, ref_bbox.centerY)
transform = ref_wcs.linearizePixelToSky(sky0,geom.arcseconds)
jacmat = transform.getLinear().getMatrix()

In [ ]:
transform.getMatrix()

In [ ]:
jacmat

In [ ]:
ref_wcs.getFitsMetadata()

In [ ]:
test = ref_wcs.getFrameDict()

In [ ]:
type(test)

In [ ]:
dir(test)

In [ ]:
from lsst.afw.geom.skyWcs import SkyWcs

In [ ]:
skyWcs = SkyWcs(test)

In [ ]:
skyWcs.getFitsMetadata()

In [ ]:
ref_wcs.getTanWcs(Point2D(20000,2000))

In [ ]:
LsstCam.

In [ ]:
expId = 2025042500591
ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera.get(ref_detName)
ref_raw = butler.get('raw', detector=ref_detector.getId(), 
                        exposure=expId, instrument=instrument)
ref_wcs = ref_raw.getWcs()
ref_bbox = ref_detector.getBBox()
ref_center_pixels = Point2D(ref_bbox.centerX, ref_bbox.centerY)

x_shifts = np.linspace(0, 350, 50) # in mm
eff_scales = []
dx = 20 # +/- shift in pixels
for x_shift in x_shifts:
    pixel_shift = x_shift * 100 # Convert mm to pixels
    pixels_1 = Point2D(ref_bbox.centerX + pixel_shift - dx, ref_bbox.centerY)
    pixels_2 = Point2D(ref_bbox.centerX + pixel_shift + dx, ref_bbox.centerY)
    sky_1 = ref_wcs.pixelToSky(pixels_1)
    sky_2 = ref_wcs.pixelToSky(pixels_2)
    separation = np.degrees(angular_separation(sky_1.getRa().asRadians(), sky_1.getDec().asRadians(),
                                sky_2.getRa().asRadians(), sky_2.getDec().asRadians())) * 3600
    eff_scale = separation / (2.0 * dx)
    print(x, separation, eff_scale)
    eff_scales.append(eff_scale)
                            


In [ ]:
fig = plt.figure(figsize=(8,5))
plt.scatter(x_shifts, eff_scales, label='Effective plate scale from R22_S11 WCS')
def func(x, quad, const):
    # Quadratic function with no linear term
    # So df/dr(r=0) = 0.
    return quad * x * x + const
popt, pcov = curve_fit(func, x_shifts, eff_scales, p0=[1E-4, 0.0])
print(popt)
xplot = np.linspace(0,350, 100)
yplot = xplot * xplot * popt[0] + popt[1]

plt.plot(xplot, yplot, color='red', ls='--', label='Fit to  effective plate scales')
plt.scatter(rs, scales, marker='x', color='green', label='Plate scales from WCS for each CCD')
plt.title(f"Effective Plate scale vs detector radius {expId}")
plt.xlabel("Radius to center of detector or evaluation point (mm)")
plt.ylabel("Effective Platescale (arcseconds/pixel)")
plt.legend(loc='lower left')
plt.savefig(f"/home/c/cslage/u/LSSTCam/data/Platescale_Effective_Variation_Raw_{expId}_23May25.png")

In [ ]:
ref_wcs.getFitsMetadata()